In [ ]:
import pandas as pd
df_label = pd.read_csv('../input/dog-breed-identification/labels.csv')
df_label.head()
label_name = df_label['breed'].sort_values().unique() # 유일한 
from sklearn.preprocessing import LabelEncoder

In [ ]:
import pandas as pd

In [ ]:
# labels 살펴보기
df_label = pd.read_csv('../input/dog-breed-identification/labels.csv')
df_label.head()

# 전략1 cnn
# 1. 이미지 read -> 벡터 (입력 텐서) -> ImageDataGenerator
# 2. 라벨 -> 1차원 벡터 


In [ ]:
# 총 몇 건일까요?
df_label.shape

In [ ]:
# breed별 분포 ( 건수 ) 종류별로 얼마나 있는지 
df_label['breed'].value_counts()

In [ ]:
# 120개 종류를 출력 

df_label['breed'].sort_values().unique() # 유일한 

In [ ]:

label_name = df_label['breed'].sort_values().unique() # 유일한 
print(label_name)

In [ ]:
# label_name 의 건수, type

print(len(label_name), type(label_name), label_name.dtype)


In [ ]:
# dog bleed string을 숫자로 변환
# label encoding (정수), one-hot encoding 
# LabelEncoder 객체 -> fit(df_label['breed']) -> transform(df_label['breed'])

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(df_label['breed'])  # 기준정보  120개 120개 - 100개 (20개)
df_label['breed'] = encoder.transform(df_label['breed'])

In [ ]:
df_label.head()

In [ ]:
df_label.info()


In [ ]:
# id 에 .jpg 파일이름으로 변환
df_label['id'] = df_label['id'] + '.jpg'
df_label.head()

In [ ]:
# ImageDataGenerator 

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale = 1/255. ,
    rotation_range = 20,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.1,
    zoom_range = 0.1,
    horizontal_flip = True,
    fill_mode='nearest',
    validation_split = 0.2    
)

# 증식 사용하면 안됨
val_datagen  = ImageDataGenerator (
    rescale = 1/255.,
    validation_split = 0.2    
)



In [ ]:
train_dir = '../input/dog-breed-identification/train'
df_label['breed'] = df_label['breed'].astype(str)

train_generator = train_datagen.flow_from_dataframe(
    dataframe = df_label,
    directory = train_dir,
    x_col = 'id'  ,      # filename
    y_col = 'breed' ,    # label정보
    target_size = (150,150),
    class_mode = 'categorical',
    batch_size = 128,
    subset = "training",
    seed = 7
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe = df_label,
    directory = train_dir,
    x_col = 'id'  ,      # filename
    y_col = 'breed' ,    # label정보
    target_size = (150,150),
    class_mode = 'categorical',
    batch_size = 128,
    subset = "validation",
    seed = 7
)

In [ ]:
d, l = next(train_generator)
l.shape

In [ ]:
df_label.head()


In [ ]:
################## 모델 생성 cnn
Sequential 객체 생성
conv layer ( filter 32, kernel size (3,3), activation 'relu' , input_shape = ???? )
pooling layer ( pool_size ( 2,2 ))
conv layer ( filter 64, kernel size (3,3), activation 'relu')
pooling layer ( pool_size ( 2,2 ))
conv layer ( filter 128, kernel size (3,3), activation 'relu')
pooling layer ( pool_size ( 2,2 ))
conv layer ( filter 128, kernel size (3,3), activation 'relu')
pooling layer ( pool_size ( 2,2 ))
flatten layer
Dropout (0.5)
Dense layer 출력 뉴런 수 512개, 활성화 함수 'relu'
Dense layer 출력 뉴런 수 ?????, 활성화 함수'????'


In [ ]:
# 모델 생성 -> compile -> fit -> predict -> submission 파일 만들기
# 전략: cnn, pretrained 사전학습된 vgg16 - 빠른 추출 (현업 batch), 일반적 방법

from tensorflow.keras import layers, models, optimizers

model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(128,(3,3),activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(128,(3,3),activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(120, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
len(train_generator)

In [ ]:
# compile -> fit -> predict -> submission 파일 만들기
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['accuracy'])

In [ ]:
# fit
history = model.fit(
      train_generator,
      steps_per_epoch=len(train_generator) - 1,
      epochs=5,
      validation_data=val_generator,
      validation_steps=len(val_generator) - 1
)

In [ ]:
# predict -> submission 파일 만들기

base_dir = '../input/dog-breed-identification/'

test_datagen = ImageDataGenerator(rescale=1/255.)

test_generator = test_datagen.flow_from_directory(base_dir,
                                                  target_size=(150,150),
                                                  batch_size=128,
                                                  classes=['test'])

In [ ]:
## predict 

preds = model.predict(test_generator)

In [ ]:
len(preds)

In [ ]:
len(test_generator)

In [ ]:
preds[-1]

In [ ]:
# submit 
df_submit = pd.read_csv('../input/dog-breed-identification/sample_submission.csv')

In [ ]:
df_submit.head()

In [ ]:
class_to_num = dict(zip(label_name, range(120))) # 알파벳순, 0~119


In [ ]:
class_to_num['affenpinscher']   # dog breed의 index값 (알파벳순 )

In [ ]:
for name in label_name:
    df_submit[name] = preds[:,class_to_num[name]]  

# name = 'affenpinscher'
# df_submit['affenpinscher'] 
# preds[:,0]          <- class_to_num['affenpinscher'] = 0

In [ ]:
df_submit.to_csv('submission_dog.csv', index=False)